In [39]:
import pandas as pd

# Carica il dataset
imdb_data = pd.read_csv('test/new_imdb.csv', delimiter=';')

# Estrai la valuta, ignorando le entrate che iniziano con un numero
imdb_data['currency'] = imdb_data['budget'].dropna().apply(lambda x: x.split()[0] if not x[0].isdigit() else None)

# Rimuovi le righe dove 'currency' è None
imdb_data = imdb_data.dropna(subset=['currency'])

# Conta quanti film ci sono per ogni valuta
currency_counts = imdb_data['currency'].value_counts()

# Salva il conteggio delle valute in un file di testo
currency_counts.to_csv('currency_counts_updated.txt', sep='\t', header=True)

print("Currency counts have been saved to 'currency_counts_updated.txt'.")



Currency counts have been saved to 'currency_counts_updated.txt'.


In [22]:
import pandas as pd

# Caricamento dei dati dei tassi di cambio
exchange_rates = pd.read_csv('test/eurofxref-hist.csv')
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'])

# Conversione delle colonne in numeri, gestendo le 'N/A' come NaN
exchange_rates.iloc[:, 1:] = exchange_rates.iloc[:, 1:].replace('N/A', None).apply(pd.to_numeric, errors='coerce')

# Calcolo dei tassi inversi da valuta a EUR e da EUR a USD
currency_to_eur = exchange_rates.set_index('Date').apply(lambda x: 1 / x)
eur_to_usd = 1 / currency_to_eur['USD']


In [23]:
# Carica i dati di IMDb
imdb_data = pd.read_csv('test/new_imdb.csv', delimiter=';')


# Funzione per correggere le date che contengono solo l'anno
def fix_date(date_str):
    # Controlla se la data è composta solo dall'anno (lunghezza 4)
    if len(date_str) == 4:
        # Aggiungi "-01-01" per fare l'anno completo al primo gennaio
        return date_str + "-01-01"
    else:
        # Se la data è già completa, la lascia inalterata
        return date_str

# Applica la funzione di correzione alla colonna della data
imdb_data['date_published'] = imdb_data['date_published'].apply(fix_date)

# Estrai valuta e importo dal campo 'budget'
def extract_currency_amount(budget):
    if pd.isna(budget) or budget[0] == '$' or budget[0].isdigit():
        return ('USD', budget.replace('$', '').replace(',', '')) if budget[0] == '$' else (None, budget)
    currency, amount = budget.split()[0], budget.split()[1]
    return (currency, float(amount.replace(',', '')))

imdb_data['currency_amount'] = imdb_data['budget'].apply(extract_currency_amount)


In [36]:
import pandas as pd

# Carica i dati dei tassi di cambio e prepara il DataFrame
exchange_rates = pd.read_csv('test/eurofxref-hist.csv')
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'])
exchange_rates.set_index('Date', inplace=True)  # Imposta 'Date' come indice

# Preparazione dei DataFrame per la conversione
currency_to_eur = exchange_rates.drop(columns=['USD']).apply(lambda x: 1 / x).ffill().bfill()  # Da valuta a EUR
eur_to_usd = exchange_rates['USD'].ffill().bfill()  # Da EUR a USD (non invertire qui)

# Funzione per trovare la data di cambio più vicina
def find_nearest_date(date, available_dates):
    if date in available_dates:
        return date
    month_dates = available_dates[available_dates.month == date.month]
    return month_dates.iloc[(month_dates - date).abs().argsort()[0]] if not month_dates.empty else available_dates.iloc[(available_dates - date).abs().argsort()[0]]

def convert_to_usd(row):
    try:
        if 'currency_amount' not in row or row['currency_amount'] is None:
            return None  # Se 'currency_amount' non è definito
        currency, amount = row['currency_amount']
        if currency is None or currency == 'USD':
            return amount  # Restituisce l'importo originale se è già in USD
        date = row['date_published'].normalize()
        nearest_date = find_nearest_date(date, currency_to_eur.index)
        if currency == 'EUR':
            amount_in_usd = float(amount) * eur_to_usd.at[nearest_date]
        else:
            amount_in_eur = float(amount) * currency_to_eur.at[nearest_date, currency]
            amount_in_usd = amount_in_eur * eur_to_usd.at[nearest_date]
        return amount_in_usd
    except Exception as e:
        print(f"Error processing row: {row}")
        print(f"Exception: {e}")
        return None

# Estrai valuta e importo dal campo 'budget'
def extract_currency_amount(budget):
    if pd.isna(budget) or budget[0] == '$' or budget[0].isdigit():
        return ('USD', budget.replace('$', '').replace(',', '')) if budget[0] == '$' else (None, budget)
    currency, amount = budget.split()[0], budget.split()[1]
    return (currency, float(amount.replace(',', '')))
# Carica i dati IMDb e applica la conversione
imdb_data = pd.read_csv('test/new_imdb.csv', delimiter=';')

# Funzione per correggere le date che contengono solo l'anno
def fix_date(date_str):
    return date_str + "-01-01" if len(date_str) == 4 else date_str

imdb_data['date_published'] = imdb_data['date_published'].apply(fix_date)
imdb_data['date_published'] = pd.to_datetime(imdb_data['date_published'])
imdb_data['currency_amount'] = imdb_data['budget'].apply(extract_currency_amount)
imdb_data['converted_budget'] = imdb_data.apply(convert_to_usd, axis=1)

# Salva i risultati
imdb_data.to_csv('updated_imdb_data_with_converted_budgets.csv', index=False)
print("bro")

Error processing row: imdb_title_id                                                    tt0011440
title                                                        Markens grøde
original_title                                               Markens grøde
year                                                                 19210
date_published                                         1921-12-02 00:00:00
genre                                                                Drama
duration                                                              1070
country                                                             Norway
language                                                               NaN
director                                                Gunnar Sommerfeldt
writer                                     Knut Hamsun, Gunnar Sommerfeldt
production_company                                         Christiana Film
actors                   Amund Rydland, Karen Poulsen, Ragna Wettergree...
des

In [37]:
import pandas as pd
import numpy as np

# Carica i dati IMDb e applica la conversione, assumendo che questo sia già stato fatto
imdb_data = pd.read_csv('updated_imdb_data_with_converted_budgets.csv')

# Assicurati che 'converted_budget' sia nel formato numerico corretto
imdb_data['converted_budget'] = pd.to_numeric(imdb_data['converted_budget'], errors='coerce')

# Aggiorna la colonna 'budget' solo dove 'converted_budget' non è NaN
imdb_data['budget'] = np.where(pd.notna(imdb_data['converted_budget']), imdb_data['converted_budget'].astype(str), imdb_data['budget'])

# Salva i risultati nel file CSV aggiornato
imdb_data.to_csv('final_imdb_data_with_updated_budgets.csv', index=False)


In [40]:
import pandas as pd
# Carica il dataset
imdb_data = pd.read_csv('final_imdb_data_with_updated_budgets.csv')

# Estrai la valuta, ignorando le entrate che iniziano con un numero
imdb_data['currency'] = imdb_data['budget'].dropna().apply(lambda x: x.split()[0] if not x[0].isdigit() else None)

# Rimuovi le righe dove 'currency' è None
imdb_data = imdb_data.dropna(subset=['currency'])

# Conta quanti film ci sono per ogni valuta
currency_counts = imdb_data['currency'].value_counts()

currency_counts.to_csv('currency_counts.txt', sep='\t', header=True)

print("Currency counts have been saved to 'currency_counts.txt'.")


Currency counts have been saved to 'currency_counts.txt'.


In [2]:
import pandas as pd


df = pd.read_csv('../dataset/final_imdb_data_with_updated_budgets.csv')

df_filtered = df[df['budget'].str.match(r'^(\$|\d)')]
df_filtered['budget'] = pd.to_numeric(df['budget'].str.replace(r'^\$ ', '', regex=True), errors='coerce')
df_filtered['worlwide_gross_income'] = pd.to_numeric(df['worlwide_gross_income'].str.replace(r'^\$ ', '', regex=True), errors='coerce')

columns_to_drop = [
    'imdb_title_id', 'original_title', 'year', 'description', 'avg_vote',
    'votes', 'usa_gross_income', 'metascore', 'reviews_from_users',
    'reviews_from_critics', 'currency_amount', 'converted_budget'
]

# Rimuovi le colonne
df_filtered.drop(columns=columns_to_drop, inplace=True)
df_filtered.to_csv('../dataset/filtered_dataset.csv', index=False)


C:\Users\BOLO\AppData\Local\Temp\ipykernel_9500\1982728625.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['budget'] = pd.to_numeric(df['budget'].str.replace(r'^\$ ', '', regex=True), errors='coerce')
C:\Users\BOLO\AppData\Local\Temp\ipykernel_9500\1982728625.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['worlwide_gross_income'] = pd.to_numeric(df['worlwide_gross_income'].str.replace(r'^\$ ', '', regex=True), errors='coerce')
C:\Users\BOLO\AppData\Local\Temp\ipykernel_9500\